In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

import re

In [2]:
df = pd.read_csv("resale-flat-prices 2/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

In [4]:
df['remaining_lease'] = df['remaining_lease'].str.slice(stop=2).astype(int)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148277 entries, 0 to 148276
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                148277 non-null  object 
 1   town                 148277 non-null  object 
 2   flat_type            148277 non-null  object 
 3   block                148277 non-null  object 
 4   street_name          148277 non-null  object 
 5   storey_range         148277 non-null  object 
 6   floor_area_sqm       148277 non-null  float64
 7   flat_model           148277 non-null  object 
 8   lease_commence_date  148277 non-null  int64  
 9   remaining_lease      148277 non-null  int64  
 10  resale_price         148277 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 12.4+ MB


### Using OneMap's API to generate the corresponding geographical information

In [9]:
import json
import requests
import urllib.parse

address = df['street_name'][4]+" "+df['block'][4]
query_address = urllib.parse.quote(address)
query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y&pageNum=1'
resp = requests.get(query_string)
        
#Convert JSON into Python Object 
data=json.loads(resp.content)

In [10]:
data

{'found': 2,
 'totalNumPages': 1,
 'pageNum': 1,
 'results': [{'SEARCHVAL': 'MY FIRST SKOOL',
   'BLK_NO': '601',
   'ROAD_NAME': 'ANG MO KIO AVENUE 5',
   'BUILDING': 'MY FIRST SKOOL',
   'ADDRESS': '601 ANG MO KIO AVENUE 5 MY FIRST SKOOL SINGAPORE 560601',
   'POSTAL': '560601',
   'X': '28201.7824866865',
   'Y': '40334.0512122996',
   'LATITUDE': '1.38104134784496',
   'LONGITUDE': '103.835131744823',
   'LONGTITUDE': '103.835131744823'},
  {'SEARCHVAL': 'YIO CHU KANG GREEN',
   'BLK_NO': '601',
   'ROAD_NAME': 'ANG MO KIO AVENUE 5',
   'BUILDING': 'YIO CHU KANG GREEN',
   'ADDRESS': '601 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SINGAPORE 560601',
   'POSTAL': '560601',
   'X': '28201.7822445426',
   'Y': '40334.0520295352',
   'LATITUDE': '1.38104135523576',
   'LONGITUDE': '103.835131742647',
   'LONGTITUDE': '103.835131742647'}]}

#### Take note that the run-time expected from the cell below is about 6h

In [16]:
import time

for i in range(df.shape[0]):
    if (df['Latitude'].isnull().iloc[i]):
        address = df['street_name'][i]+" "+df['block'][i]
        query_address = urllib.parse.quote(address)
        query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y&pageNum=1'
        resp = requests.get(query_string)

        #Convert JSON into Python Object 
        data=json.loads(resp.content)

        if(data['found'] == 0): 
            print("no data in row:", i)
        else:
            df.loc[i, 'Longitude'] = data['results'][-1]['LONGITUDE']
            df.loc[i, 'Latitude'] = data['results'][-1]['LATITUDE']
            df.loc[i, 'Address'] = data['results'][-1]['ADDRESS']
            df.loc[i, 'Postal code'] = data['results'][-1]['POSTAL']

        if((i+1)%250==0):
            print("Sleeping...", i)
            time.sleep(15)
    else: continue

Sleeping... 124499
Sleeping... 124749
Sleeping... 124999
Sleeping... 125249
Sleeping... 125499
Sleeping... 125749
Sleeping... 125999
Sleeping... 126249
Sleeping... 126499
Sleeping... 126749
Sleeping... 126999
Sleeping... 127249
Sleeping... 127499
Sleeping... 127749
Sleeping... 127999
Sleeping... 128249
Sleeping... 128499
Sleeping... 128749
Sleeping... 128999
Sleeping... 129249
Sleeping... 129499
Sleeping... 129749
Sleeping... 129999
Sleeping... 130249
Sleeping... 130499
Sleeping... 130749
Sleeping... 130999
Sleeping... 131249
Sleeping... 131499
Sleeping... 131749
Sleeping... 131999
Sleeping... 132249
Sleeping... 132499
Sleeping... 132749
Sleeping... 132999
Sleeping... 133249
Sleeping... 133499
Sleeping... 133749
Sleeping... 133999
Sleeping... 134249
Sleeping... 134499
Sleeping... 134749
Sleeping... 134999
Sleeping... 135249
Sleeping... 135499
Sleeping... 135749
Sleeping... 135999
Sleeping... 136249
Sleeping... 136499
Sleeping... 136749
Sleeping... 136999
Sleeping... 137249
Sleeping... 

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148277 entries, 0 to 148276
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                148277 non-null  object 
 1   town                 148277 non-null  object 
 2   flat_type            148277 non-null  object 
 3   block                148277 non-null  object 
 4   street_name          148277 non-null  object 
 5   storey_range         148277 non-null  object 
 6   floor_area_sqm       148277 non-null  float64
 7   flat_model           148277 non-null  object 
 8   lease_commence_date  148277 non-null  int64  
 9   remaining_lease      148277 non-null  int64  
 10  resale_price         148277 non-null  float64
 11  Longitude            148277 non-null  object 
 12  Latitude             148277 non-null  object 
 13  Address              148277 non-null  object 
 14  Postal code          148277 non-null  object 
dtypes: float64(2), in

In [19]:
df = df[df['Latitude'].isnull()==False]
df.to_csv('df_with_latlong.csv',index=False)

In [20]:
df[df['Postal code'] == "NIL"]
# Might have to remove these rows, unless postal code will not be used in our model

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Longitude,Latitude,Address,Postal code
41,2017-01,ANG MO KIO,4 ROOM,463,ANG MO KIO AVE 10,04 TO 06,92.0,New Generation,1980,62,440000.0,103.85713855795,1.36711621129028,463 ANG MO KIO AVENUE 10,NIL
135,2017-01,BISHAN,4 ROOM,115,BISHAN ST 12,07 TO 09,84.0,Simplified,1986,68,533000.0,103.848613166647,1.34794675925829,115 BISHAN STREET 12,NIL
182,2017-01,BUKIT BATOK,4 ROOM,386,BT BATOK WEST AVE 5,13 TO 15,90.0,Model A,2004,86,440000.0,103.752720364689,1.36399534082987,386 BUKIT BATOK WEST AVENUE 5,NIL
227,2017-01,BUKIT MERAH,4 ROOM,129,KIM TIAN RD,10 TO 12,88.0,Improved,1976,58,550000.0,103.828856188849,1.28115303287131,129 KIM TIAN ROAD,NIL
401,2017-01,GEYLANG,3 ROOM,36,CIRCUIT RD,13 TO 15,72.0,Standard,1971,53,338000.0,103.88692527127,1.32905963512054,36 CIRCUIT ROAD,NIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147799,2023-02,YISHUN,5 ROOM,322,YISHUN CTRL,01 TO 03,122.0,Improved,1988,64,558000.0,103.840831142054,1.42876366770119,322 YISHUN CENTRAL,NIL
147820,2023-03,ANG MO KIO,4 ROOM,621,ANG MO KIO AVE 9,04 TO 06,92.0,New Generation,1980,56,486000.0,103.840669101591,1.38302875418539,621 ANG MO KIO AVENUE 9,NIL
148027,2023-03,KALLANG/WHAMPOA,3 ROOM,74,WHAMPOA DR,10 TO 12,66.0,Improved,1981,57,365000.0,103.857783454168,1.32472597938151,74 WHAMPOA DRIVE,NIL
148261,2023-03,YISHUN,4 ROOM,412,YISHUN RING RD,01 TO 03,104.0,Model A,1993,68,450000.0,103.846968481236,1.42574075059106,412 YISHUN RING ROAD,NIL
